In [72]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import time
import re
from tqdm import tqdm
import timeit
tqdm.pandas()
import omap


C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
import pandas as pd
import datetime
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statistics import mean 
import scipy.signal
print(scipy.__version__)
from scipy.signal import find_peaks, peak_prominences
import collections 
  
sns.set(rc={'figure.figsize':(20, 10)})

# omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_45_20191114_1800_to_20191114_1900.csv'
# evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv'

omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_46_20191114_1700_to_20191114_2000.csv'
evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_2000.csv'

# omap_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/OMAP_Train_25_Car_49_20210111_2000_to_20210111_2359.csv'
# evr_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/EVR_20210111_2000_to_20210111_2359.csv'

# omap_output_filename = './2017_Train18_20170810/preprocessing_output/OMAP_Train_18_Car_35_20170810_0500_to_20170810_1000.csv'
# evr_output_filename = './2017_Train18_20170810/preprocessing_output/EVR_20170810_0500_to_20170810_1000.csv'

#Import OMAP & EVR
omap_df = pd.read_csv(omap_output_filename)
# Find peak speed in ATP
ATO_peak_timestamp = omap_df.loc[omap_df['ATO_0803_Train_speed'].idxmax()]['epoch']
print(omap_df.loc[omap_df['ATP_026_Raw_speed'].idxmax()]['epoch'], omap_df.loc[omap_df['ATP_026_Raw_speed'].idxmax()]['ATP_026_Raw_speed'])
print(omap_df.iloc[0]['epoch'], omap_df.iloc[-1]['epoch'])
difference_duration = datetime.datetime.fromtimestamp((omap_df.iloc[-1]['epoch']-omap_df.iloc[0]['epoch']) // 1000).strftime('%Y-%m-%d %H:%M:%S')
print(difference_duration)

evr_df = pd.read_csv(evr_output_filename)
# Find peak speed in EOD
EOD_peak_timestamp = evr_df.loc[evr_df['EOD_179_V1_OA_DMASPE'].idxmax()]['epoch']
print(evr_df.loc[evr_df['EOD_179_V1_OA_DMASPE'].idxmax()]['epoch'], evr_df.loc[evr_df['EOD_179_V1_OA_DMASPE'].idxmax()]['EOD_179_V1_OA_DMASPE'])
print(evr_df.iloc[0]['epoch'], evr_df.iloc[-1]['epoch'])
y_difference_duration = datetime.datetime.fromtimestamp((evr_df.iloc[-1]['epoch']-evr_df.iloc[0]['epoch']) // 1000).strftime('%Y-%m-%d %H:%M:%S')
print(y_difference_duration)


#evr_df['epoch_mod'] = evr_df['epoch'].apply(lambda x: x-19500) #19000
x = omap_df['ATO_0803_Train_speed'].interpolate().values
y = evr_df['EOD_179_V1_OA_DMASPE'].interpolate().values

# Plot Speed Profile
plt.figure(0)
plt.plot(omap_df['epoch'], omap_df['ATO_0803_Train_speed'].interpolate().values, label="ATO_0803_Train_speed")
plt.plot(omap_df['epoch'], omap_df['ATP_026_Raw_speed'].interpolate().values, label="ATP_026_Raw_speed")
plt.plot(evr_df['epoch'], evr_df['EOD_179_V1_OA_DMASPE'].interpolate().values, label="EOD_179_V1_OA_DMASPE")
plt.xlabel('Epoch')
plt.ylabel('Raw Speed')
plt.legend(loc="upper left")
plt.title("OMAP & EVR Raw Speed Profile")
plt.show()

# Plot Peak Speed Profile
print(ATO_peak_timestamp, EOD_peak_timestamp)
difference = ATO_peak_timestamp-EOD_peak_timestamp
print('Difference:', difference)
plt.figure(1)
plt.plot(omap_df['epoch'], x, label="ATO_0803_Train_speed")
plt.plot(evr_df['epoch'], y, label="EOD_179_V1_OA_DMASPE",color='green')
plt.scatter(ATO_peak_timestamp,  omap_df.loc[omap_df['ATO_0803_Train_speed'].idxmax()]['ATO_0803_Train_speed'], color='blue')
plt.scatter(EOD_peak_timestamp,  evr_df.loc[evr_df['EOD_179_V1_OA_DMASPE'].idxmax()]['EOD_179_V1_OA_DMASPE'],color='green')
plt.xlabel('Epoch')
plt.ylabel('Raw Speed')
plt.legend(loc="upper left")
plt.title("Omap & Evr Raw Speed Profile")
plt.show()

def top_x_prominences(speed, top_num_candidate, name):
    if name == "OMAP":
        distance = 1000
    elif name == "EVR":
        distance = 200
    # Find peak
    peaks, properties = find_peaks(speed, distance=distance)
    
    # Find top x prominences
    prominences = peak_prominences(speed, peaks)[0]
    # Sort Descending and retrieve top x
    top_5_prominences_index =  prominences.argsort()[-top_num_candidate:][::-1]
    
    contour_heights = speed[peaks] - prominences
    
    print(peaks, contour_heights, top_5_prominences_index)
    return peaks, contour_heights, top_5_prominences_index

def estimate_time_shift(omap_df, evr_df):
    omap_df_speed = omap_df['ATO_0803_Train_speed'].interpolate().values
    evr_df_speed = evr_df['EOD_179_V1_OA_DMASPE'].interpolate().values
    top_num_candidate = 10
    # Find OMAP Speed Peak Prominences
    omap_peaks, omap_contour_heights, omap_top_x_prominences_index = top_x_prominences(omap_df_speed, top_num_candidate, "OMAP")
    # Find EVR Speed Peak Prominences
    evr_peaks, evr_contour_heights, evr_top_x_prominences_index = top_x_prominences(evr_df_speed, top_num_candidate, "EVR")
        
    # Find differences between peak prominences
    differences_list = []
    for index in omap_top_x_prominences_index:
        omap_timestamp = omap_df.iloc[omap_peaks[index]]['epoch']
        for y_index in evr_top_x_prominences_index:
            evr_timestamp = evr_df.iloc[evr_peaks[y_index]]['epoch']
            diff = omap_timestamp-evr_timestamp
            if abs(diff) <= 25000:
                differences_list.append(diff)
    
    differences_list_div = np.divide(differences_list, 1000).astype(int)
    
    
    occurence_count = collections.Counter(differences_list_div) 
    most_common = occurence_count.most_common(1)[0][0]
    index_list = []
    if 0 in differences_list_div:
        #Find index
        index_list = [i for i, j in enumerate(differences_list_div) if j == 0]
    else:
        index_list = [i for i, j in enumerate(differences_list_div) if j != 0]
    
    print(differences_list_div)
    print(index_list)
    indexed_differences_list = map(lambda i: differences_list[i], index_list)
    mean_time_shift = mean(indexed_differences_list)
    print(differences_list, mean_time_shift)
    
    plt.figure()
    plt.plot(omap_df['epoch'].values, omap_df_speed)
    plt.plot(evr_df['epoch'].values + mean_time_shift, evr_df_speed)
    plt.show()
    
    fig, axs = plt.subplots(nrows=3,sharex=True,sharey=True)
    fig.subplots_adjust(hspace=0.7)
    ax = axs[0]
    ax.plot(omap_df['epoch'].values, omap_df_speed, 'b')
    ax.plot(omap_df['epoch'].values[omap_peaks], omap_df_speed[omap_peaks], 'gx')
    ax.plot(omap_df['epoch'].values[omap_peaks[omap_top_x_prominences_index]], omap_df_speed[omap_peaks[omap_top_x_prominences_index]], 'kx')
    ax.vlines(x=omap_df['epoch'].values[omap_peaks[omap_top_x_prominences_index]], ymin=omap_contour_heights[omap_top_x_prominences_index], ymax=omap_df_speed[omap_peaks[omap_top_x_prominences_index]], color='k')
    ax.set_title ("OMAP Peaks and Top {} Prominences" .format(top_num_candidate))
    top_str = "Top {} Prominences" .format(top_num_candidate)
    ax.legend(["Raw Speed", "Peaks", top_str], loc="upper left")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Raw Speed')
    ax.grid(True)
    
    ax = axs[1]
    ax.plot(evr_df['epoch'].values, evr_df_speed, 'r')
    ax.plot(evr_df['epoch'].values[evr_peaks], evr_df_speed[evr_peaks], 'gx')
    ax.plot(evr_df['epoch'].values[evr_peaks[evr_top_x_prominences_index]], evr_df_speed[evr_peaks[evr_top_x_prominences_index]], 'kx')
    ax.vlines(x=evr_df['epoch'].values[evr_peaks[evr_top_x_prominences_index]], ymin=evr_contour_heights[evr_top_x_prominences_index], ymax=evr_df_speed[evr_peaks[evr_top_x_prominences_index]], color='k')
    ax.set_title ("OMAP Peaks and Top {} Prominences" .format(top_num_candidate))
    top_str = "Top {} Prominences" .format(top_num_candidate)
    ax.legend(["Raw Speed", "Peaks", top_str], loc="upper left")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Raw Speed')
    ax.grid(True)
    
    ax = axs[2]
    ax.plot(omap_df['epoch'].values, omap_df_speed, 'b')
    ax.plot(evr_df['epoch'].values, evr_df_speed, 'r')
    ax.plot(evr_df['epoch'].values + mean_time_shift, evr_df_speed, 'g')
    ax.set_title ("OMAP & EVR Time Syncrhonized")
    ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE", "EOD_179_V1_OA_DMASPE_Corrected"], loc="upper left")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Raw Speed')
    ax.grid(True)
    
    return mean_time_shift
    
time_shift = estimate_time_shift(omap_df, evr_df)

# omap_df = omap_df.set_index('epoch')
# evr_df = evr_df.set_index('epoch')
# #evr_df_2 = evr_df.set_index('epoch_mod')



# # Normalize column values
# # omap_df['ATP_026_Raw_speed_NORM'] = omap_df['ATP_026_Raw_speed'] / omap_df['ATP_026_Raw_speed'].max()
# # evr_df['EOD_179_V1_OA_DMASPE_NORM'] = evr_df['EOD_179_V1_OA_DMASPE'] / evr_df['EOD_179_V1_OA_DMASPE'].max()
# # omap_df['ATP_026_Raw_speed_NORM'].interpolate(method='linear').plot()
# # evr_df['EOD_179_V1_OA_DMASPE_NORM'].interpolate(method='linear').plot()

# plt.figure(6)
# omap_df['ATP_026_Raw_speed'].interpolate(method='linear').plot()
# evr_df['EOD_179_V1_OA_DMASPE'].interpolate(method='linear').plot()
# #evr_df_2['EOD_179_V1_OA_DMASPE'].interpolate(method='linear').plot()

# #plt.plot(omap_df['epoch'], omap_df['ATP_026_Raw_speed'],'o-')
# #print(omap_df['ATP_026_Raw_speed'])

# plt.scatter(ATP_peak_timestamp,  omap_df.loc[omap_df['ATP_026_Raw_speed'].idxmax()]['ATP_026_Raw_speed'], color='blue')
# plt.scatter(EOD_peak_timestamp,  evr_df.loc[evr_df['EOD_179_V1_OA_DMASPE'].idxmax()]['EOD_179_V1_OA_DMASPE'],color='red')
# plt.xlabel('Epoch')
# plt.ylabel('Raw Speed')
# #plt.legend()
# plt.show()


Using matplotlib backend: Qt5Agg
1.6.0
1573755685200.0 82.62
1573747116000.0 1573757893858.0
1970-01-01 10:59:37
1573755704000.0 82.0
1573750800000.0 1573757057000.0
1970-01-01 09:44:17
1573755685200.0 1573755704000.0
Difference: -18800.0
[ 51210  52306  53607  55121  64177  65594  66601  69852  71474  73402
  75382  77308  79727  81378  83114  85039  86880  88432  90377  92328
  93828  95786  96788  99265 101086 102101 104010 105661 107466 109193
 110979 112646 114458 116522 117901 120435 122060 124015 126109 127760
 131991 134352 136284 138282 140218 142628 144331 146029 148080 149872
 151456] [ 1.62 13.72 13.68  0.    0.    0.   50.47  0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   57.53  0.
  0.   68.08  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   50.83] [35 21 43  7  5 36 24 10 32 34]
[ 148  385 1192 1606 1968 2186 2409 2947 3330 3646 4150 4439 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\eventloops.py:106: MatplotlibDeprecationWarning: Toggling axes navigation from the keyboard is deprecated since 3.3 and will be removed two minor releases later.
  app.exec_()


In [9]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
# Cross Correlation
# Set parameters

x = 0.28328
n = 1000
n_repeats = 5

# Get correlations
t = np.linspace(0, n_repeats, n)
time = np.arange(1-n,n)

sin_delay = lambda delay: np.sin(2.0 * np.pi * (t - delay))

# omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_45_20191114_1800_to_20191114_1900.csv'
# evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv'

omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_46_20191114_1700_to_20191114_2000.csv'
evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_2000.csv'

# omap_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/OMAP_Train_25_Car_49_20210111_2000_to_20210111_2359.csv'
# evr_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/EVR_20210111_2000_to_20210111_2359.csv'

omap_df = pd.read_csv(omap_output_filename)
evr_df = pd.read_csv(evr_output_filename)

x = omap_df['ATO_0803_Train_speed'].interpolate().values
y = evr_df['EOD_179_V1_OA_DMASPE'].interpolate().values
x_epoch = omap_df['epoch'].values
y_epoch = evr_df['epoch'].values

lags = signal.correlation_lags(x.size, y.size, mode="full")

# signal1 = sin_delay(delay=0)
# signal2 = sin_delay(delay=x)

signal1 = x
signal2 = y

corr12 = signal.correlate(signal1, signal2, mode = 'full')

a2 = np.argmax(corr12)

fig, axs = plt.subplots(nrows=3)
fig.subplots_adjust(hspace=0.4)
ax = axs[0]
ax.plot(x_epoch, signal1, "r")
ax.plot(y_epoch, signal2, "b")
#x.set_title ("Signals, delay = {:.3f}".format(x))
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE"], loc="upper left")
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.grid(True)

ax = axs[1]
ax.plot(lags, corr12, "b")
ax.set_title("Cross-Correlations")
ax.set_xlabel('Samples')
ax.grid(True)
ax.scatter(lags[corr12.argmax()],  corr12[corr12.argmax()], color='blue')

dx = np.mean(np.diff(x_epoch))
maxlag = lags[corr12.argmax()]*1.0
#maxlag = time[corr12.argmax()]*1.0* n_repeats/n
print("max correlation is at lag %f" % maxlag)
ax = axs[2]
ax.plot(x_epoch, signal1, "r")
ax.plot(y_epoch, signal2, "b")
ax.plot(y_epoch-maxlag, signal2, "g")
ax.set_title("Signals, delay estimated = {:.3f}".format(maxlag))
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE", "EOD_179_V1_OA_DMASPE_Corrected"], loc="upper left")
ax.grid(True)

# # Narrow down
# omap_df_narrow = omap_df.loc[(omap_df['epoch'] >= evr_df.iloc[0]['epoch']) & (omap_df['epoch'] <= evr_df.iloc[-1]['epoch'])]

# x = omap_df['ATP_026_Raw_speed'].interpolate().values
# y = evr_df['EOD_179_V1_OA_DMASPE'].interpolate().values

# correlation = scipy.signal.correlate(x, y, mode="full")
# lags = scipy.signal.correlation_lags(x.size, y.size, mode="full")
# lag = lags[np.argmax(correlation)]
# print("Lag ",lag,np.mean(lags))

# x_epoch = omap_df_narrow['epoch'].values
# y_epoch = evr_df['epoch'].values

# dx = np.mean(np.diff(x_epoch))
# shift = (np.argmax(scipy.signal.correlate(x, y)) - len(y))*dx
# print(shift)
    
# plt.figure(4)
# plt.plot(x_epoch, x)
# plt.plot(y_epoch + mean(differences_list), y)
# plt.show()

max correlation is at lag 136107.000000


In [123]:
import fastdtw
from scipy.spatial.distance import euclidean

distance, path = fastdtw.fastdtw(omap_df['ATP_026_Raw_speed'].dropna(), evr_df['EOD_179_V1_OA_DMASPE'].dropna(), dist=euclidean)

# print(distance)
# print(path)

omap_speed = omap_df['ATP_026_Raw_speed'].dropna().tolist()
evr_speed = evr_df['EOD_179_V1_OA_DMASPE'].dropna().tolist()
#print(evr_speed)
omap_list_index = []
omap_list_mapped = []
evr_list_mapped = []

for [map_x, map_y] in path:
    omap_list_index.append(map_x)
    omap_list_mapped.append(omap_speed[map_x])
    evr_list_mapped.append(evr_speed[map_y])
    
#print(omap_list_index)

plt.figure(2)
#omap_df['ATP_026_Raw_speed'].dropna().plot()
plt.plot(omap_list_mapped,'-')
plt.plot(evr_list_mapped,'-')
plt.xlabel('Epoch')
plt.ylabel('Raw Speed')
#plt.legend()
plt.show()

In [ ]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import numpy as np
s1 = omap_df['ATP_026_Raw_speed'].dropna().to_numpy(dtype=np.double)
s2 = evr_df['EOD_179_V1_OA_DMASPE'].dropna().to_numpy(dtype=np.double)
print(s1.shape, s2.shape)
path = dtw.warping_path_fast(s1, s2)
print(len(path))
print(path)
#dtwvis.plot_warping(s1, s2, path, filename="warp.png")
#distance, path1 = dtw.warping_paths_fast(s1, s2, window=20, psi=2)
paths = dtw.warping_path_fast(s1, s2, psi=1)
print(paths)
dtwvis.plot_warping(s1, s2, paths, filename="warp.png")

omap_speed = omap_df['ATP_026_Raw_speed'].dropna().tolist()
evr_speed = evr_df['EOD_179_V1_OA_DMASPE'].dropna().tolist()
#print(evr_speed)
omap_list_index = []
omap_list_mapped = []
evr_list_mapped = []

for [map_x, map_y] in paths:
    omap_list_index.append(map_x)
    omap_list_mapped.append(omap_speed[map_x])
    evr_list_mapped.append(evr_speed[map_y])
    

plt.figure(3)
plt.plot(omap_list_mapped,'-')
plt.plot(evr_list_mapped,'-')
plt.xlabel('Epoch')
plt.ylabel('Raw Speed')
plt.show()

In [6]:
import pandas as pd
import datetime
import numpy as np
import collections
from statistics import mean
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20, 10)})
## A noisy sine wave as query
# idx = np.linspace(0,6.28,num=100)
# query = np.sin(idx) + np.random.uniform(size=100)/10.0

# omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_45_20191114_1800_to_20191114_1900.csv'
# evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv'

omap_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_46_20191114_1700_to_20191114_2000.csv'
evr_output_filename = './Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_2000.csv'

# omap_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/OMAP_Train_25_Car_50_20210111_2000_to_20210111_2359.csv'
# evr_output_filename = './Train 25 11JAN2021 Work Folder/preprocessing_output/EVR_20210111_2000_to_20210111_2359.csv'

omap_df = pd.read_csv(omap_output_filename)
evr_df = pd.read_csv(evr_output_filename)

# Narrow down
omap_df_narrow = omap_df.loc[(omap_df['epoch'] >= evr_df.iloc[0]['epoch']) & (omap_df['epoch'] <= evr_df.iloc[-1]['epoch'])]

template = omap_df_narrow['ATO_0803_Train_speed'].interpolate().to_numpy(dtype=np.double)
query = evr_df['EOD_179_V1_OA_DMASPE'].interpolate().to_numpy(dtype=np.double)
print(query.shape)

x_epoch = omap_df_narrow['epoch'].values
y_epoch = evr_df['epoch'].values

## Find the best match with the canonical recursion formula
from dtw import *
alignment = dtw(query, template, keep_internals=True)
print(template.shape, len(template))
print(query.shape, len(query[alignment.index1]))

fig, axs = plt.subplots(nrows=4)
fig.subplots_adjust(hspace=0.7)
ax = axs[0]
ax.plot(x_epoch, template, "r")
ax.plot(y_epoch, query, "b")
ax.set_title ("Roughly Similar Begin and End Points")
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE"], loc="upper left")
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.grid(True)

ax = axs[1]
ax.plot(x_epoch, template, "r")
ax.plot(x_epoch[alignment.index2],query[alignment.index1],"b")
ax.set_title ("Dynamic Time Warping")
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE_Warped"], loc="upper left")
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.grid(True)

alignment = dtw(query, template, open_end=True, keep_internals=True)
ax = axs[2]
ax.plot(x_epoch, template, "r")
ax.plot(x_epoch[alignment.index2],query[alignment.index1],"b")
ax.set_title ("Dynamic Time Warping with Open End Relieved")
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE_Warped"], loc="upper left")
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.grid(True)

alignment = dtw(query, template, step_pattern=asymmetric, open_begin=True, open_end=True, keep_internals=True)
ax = axs[3]
ax.plot(x_epoch, template, "r")
ax.plot(x_epoch[alignment.index2],query[alignment.index1],"b")
ax.set_title ("Dynamic Time Warping with Open Begin & End Relieved")
ax.legend(["ATO_0803_Train_speed", "EOD_179_V1_OA_DMASPE_Warped"], loc="upper left")
ax.set_xlabel('Time')
ax.set_ylabel('Raw Speed')
ax.grid(True)
#plt.plot(template);
#plt.plot(alignment.index2,query[alignment.index1])
plt.show()

count_dict=collections.Counter(alignment.index1)
print(mean(count_dict.values()))

## Display the warping curve, i.e. the alignment curve
#alignment.plot(type="threeway")

## Align and plot with the Rabiner-Juang type VI-c unsmoothed recursion
# dtw(query, template, keep_internals=True, 
#     step_pattern=rabinerJuangStepPattern(6, "c"))\
#     .plot(type="twoway",offset=-2)


## See the recursion relation, as formula and diagram
#print(rabinerJuangStepPattern(6,"c"))
#rabinerJuangStepPattern(6,"c").plot()

Using matplotlib backend: Qt5Agg
(11992,)


ValueError: No warping path found compatible with the local constraints